In [1]:
from prep_data import Data_prepare
import numpy as np 
# import prep_data

###  Data preprocessing 
1. covert mat data to numpy array 
2. combine male and female data to one array 
3. if needed, remove certain rows 
4. get postive and negative of Xs 
    - order difference or net vs neg 
5. reshape each X: 128*8*2 = 2048 
    - input: data 
6. generate labels for x 
    - input: data and label_type (0:zeros, 1: ones )
7. combine net and neg 
    - input: netX,netY,negX,negY

In [2]:
dp = Data_prepare() 
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
# netural vs postive 
# pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
# neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]] 
m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat" 
f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
index=[40,41]
df = dp.combine_male_female(m_file,f_file)
df = dp.remove_person(df,index)
pos_df = dp.get_pos_or_neg(df,pos)
neg_df = dp.get_pos_or_neg(df,neg)
pos_sequeezed = dp.squeeze_feature_size(pos_df)
neg_sequeezed = dp.squeeze_feature_size(neg_df)
pos_labels = dp.generate_labels(pos_sequeezed,1)
neg_labels = dp.generate_labels(neg_sequeezed,0)
x,y = dp.combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)

In [3]:
## (40,) -> 40*36*2048
def flatten(data):
    res = data[0]
    for i in range(1,data.shape[0]):
        res = np.append(res,data[i],axis=0)
    return res 
new_x = flatten(x)
new_y = flatten(y)

In [4]:
print(new_x.shape,new_y.shape)


(1404, 2048) (1404, 1)


#### Normalize x with standardScaler

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(new_x)
new_x = scaler.transform(new_x)

In [7]:
print(new_x.shape,new_y.shape)

(1404, 2048) (1404, 1)


In [22]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, test_size = 0.1, random_state = 4)

In [23]:
print(x_train.shape,y_train.shape)

(1263, 2048) (1263, 1)


In [24]:
## reshape x_train and x_test to 3D for LSTM 
x_train = np.reshape(x_train, (x_train.shape[0],1,new_x.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0],1,new_x.shape[1]))

In [25]:
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [26]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(1263, 1, 2048) (141, 1, 2048)
(1263, 1) (141, 1)


#### LSTM model 

In [36]:
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(128, input_shape=(1,2048),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,activation="sigmoid",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16,activation="sigmoid"))
model.add(Dropout(0.2))
# model.add(LSTM(50))
# model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
from keras.optimizers import SGD
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 64)             540928    
_________________________________________________________________
dropout (Dropout)            (None, 1, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 32)             12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 32)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 16)                3136      
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3

In [37]:
history = model.fit(x_train, y_train, epochs = 100, validation_data= (x_test, y_test))
score, acc = model.evaluate(x_test, y_test)

Epoch 1/100
40/40 [==============================] - 1s 15ms/step - loss: 0.7046 - accuracy: 0.5234 - val_loss: 0.6792 - val_accuracy: 0.5674
Epoch 2/100
40/40 [==============================] - 0s 6ms/step - loss: 0.6611 - accuracy: 0.6176 - val_loss: 0.6444 - val_accuracy: 0.6738
Epoch 3/100
40/40 [==============================] - 0s 6ms/step - loss: 0.6289 - accuracy: 0.6675 - val_loss: 0.6228 - val_accuracy: 0.6738
Epoch 4/100
40/40 [==============================] - 0s 5ms/step - loss: 0.6063 - accuracy: 0.7086 - val_loss: 0.6100 - val_accuracy: 0.7021
Epoch 5/100
40/40 [==============================] - 0s 5ms/step - loss: 0.5837 - accuracy: 0.7197 - val_loss: 0.6054 - val_accuracy: 0.6879
Epoch 6/100
40/40 [==============================] - 0s 5ms/step - loss: 0.5740 - accuracy: 0.7237 - val_loss: 0.5966 - val_accuracy: 0.7163
Epoch 7/100
40/40 [==============================] - 0s 5ms/step - loss: 0.5649 - accuracy: 0.7237 - val_loss: 0.5890 - val_accuracy: 0.7163
Epoch 8/100


Epoch 59/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3383 - accuracy: 0.8765 - val_loss: 0.6288 - val_accuracy: 0.7376
Epoch 60/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3408 - accuracy: 0.8860 - val_loss: 0.6430 - val_accuracy: 0.7447
Epoch 61/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3431 - accuracy: 0.8773 - val_loss: 0.6484 - val_accuracy: 0.7305
Epoch 62/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3234 - accuracy: 0.8812 - val_loss: 0.6565 - val_accuracy: 0.7518
Epoch 63/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3193 - accuracy: 0.8844 - val_loss: 0.6811 - val_accuracy: 0.7305
Epoch 64/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3174 - accuracy: 0.8781 - val_loss: 0.6772 - val_accuracy: 0.7376
Epoch 65/100
40/40 [==============================] - 0s 5ms/step - loss: 0.3169 - accuracy: 0.8884 - val_loss: 0.6767 - val_accuracy: 0.7305
Epoch 

In [39]:
%load_ext tensorboard

In [44]:
tensorboard --port=6007 --logdir runs

ERROR: Failed to launch TensorBoard (exited with 255).
Contents of stderr:
2021-03-20 18:47:41.748436: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-03-20 18:47:41.748497: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
E0320 18:47:44.021711 140058372048704 program.py:312] TensorBoard could not bind to port 6007, it was already in use
ERROR: TensorBoard could not bind to port 6007, it was already in use